Note: a continuation from Histograms_sentences_scores_feature_selection.ipynb

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [2]:
import pickle
import datetime
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [3]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [4]:
features = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/features.p', 'rb'))
labels = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/labels.p', 'rb'))

In [5]:
#get feature weights for each sentence score from 0 to 1 in bins of 10
clf = ExtraTreesClassifier()
clf = clf.fit(features, labels)
weights = clf.feature_importances_  
#Normalize so all are greater than 1
weights = weights * 100
print weights

[ 30.10172574   4.54135379   3.89958616   3.5142616    3.63725996
   3.78929883   3.66782008   3.77067032   4.92372908  38.15429444]


In [6]:
#is the issue that the ET classifier used is fit on the test data? I only want the weights, not the model itself.

In [7]:
#Apply to the actual sentence scores
a = datetime.datetime.now().replace(microsecond=0)

probssentences['0_weighted'] = 9.99
probssentences['1_weighted'] = 9.99

for i in range(len(probssentences)):
    valneg = probssentences.iloc[i][0]
    inxneg = 99999
    if 0 <= valneg < 0.1:
        inxneg = 0
    elif 0.1 <= valneg < 0.2:
        inxneg = 1
    elif 0.2 <= valneg < 0.3:
        inxneg = 2
    elif 0.3 <= valneg < 0.4:
        inxneg = 3
    elif 0.4 <= valneg < 0.5:
        inxneg = 4
    elif 0.5 <= valneg < 0.6:
        inxneg = 5
    elif 0.6 <= valneg < 0.7:
        inxneg = 6
    elif 0.7 <= valneg < 0.8:
        inxneg = 7
    elif 0.8 <= valneg < 0.9:
        inxneg = 8
    elif 0.9 <= valneg <= 1.0:
        inxneg = 9
    probssentences.set_value(i, '0_weighted', valneg * weights[inxneg])
    
    valpos = probssentences.iloc[i][1]
    inxpos = 9.9
    if 0 <= valpos < 0.1:
        inxpos = 0
    elif 0.1 <= valpos < 0.2:
        inxpos = 1
    elif 0.2 <= valpos < 0.3:
        inxpos = 2
    elif 0.3 <= valpos < 0.4:
        inxpos = 3
    elif 0.4 <= valpos < 0.5:
        inxpos = 4
    elif 0.5 <= valpos < 0.6:
        inxpos = 5
    elif 0.6 <= valpos < 0.7:
        inxpos = 6
    elif 0.7 <= valpos < 0.8:
        inxpos = 7
    elif 0.8 <= valpos < 0.9:
        inxpos = 8
    elif 0.9 <= valpos <= 1.0:
        inxpos = 9
    probssentences.set_value(i, '1_weighted', valpos * weights[inxpos])
    
b = datetime.datetime.now().replace(microsecond=0)
print b-a

0:00:44


In [7]:
print probssentences[0:20]

             0            1 doc  0_weighted  1_weighted
0       0.5986       0.4014   0    2.090120    1.371669
1   0.00792023      0.99208   0    0.286979   32.462023
2     0.369248     0.630752   0    1.288374    2.317645
3     0.445318     0.554682   0    1.521748    1.936771
4     0.999581  0.000419095   0   32.707468    0.015185
5   0.00449226     0.995508   1    0.162771   32.574188
6     0.918007    0.0819928   1   30.038278    2.970906
7      0.55223      0.44777   1    1.928212    1.530124
8     0.986808    0.0131918   1   32.289531    0.477988
9     0.721142     0.278858   1    2.790239    1.073478
10    0.111639     0.888361   1    0.524421    4.047701
11    0.717194     0.282805   1    2.774965    1.088675
12   0.0193013     0.980699   2    0.699358   32.089618
13   0.0159692     0.984031   2    0.578622   32.198651
14   0.0224181     0.977582   2    0.812290   31.987635
15    0.986999    0.0130013   2   32.295764    0.471085
16   0.0593864     0.940614   3    2.151792   30

In [1]:
prob = probssentences.groupby("doc").mean()
print prob[0:10]

NameError: name 'probssentences' is not defined

In [13]:
preds = np.ones((prob.shape[0]))
print preds.shape

(12500,)


In [14]:
preds[np.where(prob.iloc[:,0] >  prob.iloc[:,1])] = 0 # The first column is the negative model

In [15]:
print preds.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [16]:
print np.size(np.where(preds == test1["sentiment"]))*1./np.size(preds)

0.86848


In [15]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.868160030313


Especially considering the weights are fit from the test data itself, we would expect to see a big increase in performance, which we're not getting.

Why? :/ could have something to do with the fact that we fit a classifier on the test data which is pretty wacky. Or could be the way the weights were applied to the sentence scores (unlikely). 